# Projeto de Web Scraping com Python


Este projeto tem como objetivo demonstrar um web scraper em Python para coletar dados financeiros de ações da bolsa de valores e lista-los em tabelas. Utilizaremos as bibliotecas pandas, requests e BeautifulSoup para obter os dados históricos de preços de ações do site Yahoo Finance.

## Funcionalidades

- Importação de uma base de ações a partir de um arquivo Excel.
- Construção das URLs para acesso aos dados no Yahoo Finance.
- Captura das páginas web e transformação em objetos BeautifulSoup.
- Extração de métricas financeiras e dividendos das páginas web.
- Armazenamento das métricas e dividendos em DataFrames pandas.
- Tratamento de erros e armazenamento de informações rejeitadas.
- Estatísticas básicas dos dados

## Requisitos

Para executar o projeto, é necessário ter as seguintes bibliotecas:

- pandas
- request, 
- BeautifulSoup
- datetime

Você pode instalar as bibliotecas necessárias executando o seguinte comando:

```python
pip install pandas requests beautifulsoup4 datetime

In [1]:
#Imports necessários
from datetime import datetime #Tempo e data
IP=datetime.now()
import pandas as pd # Manipulação de tabelas
import requests # Captura do texto HTML do site
from bs4 import BeautifulSoup # Processamento do HTML do site

import matplotlib.pyplot as plt #gráficos

## Extração dos dados e pré-processamento.



Aqui, importamos a base de ações a partir de um arquivo Excel chamado "Ações.xlsx" utilizando a função read_excel da biblioteca pandas.

Na sequência, será adicionado nome da ação no URL pois ações brasileiras terminam com ".SA". 

O texto da coluna "URL" foi retirado da tabela de métricas de ações no site do Yahoo Finance. 

A coluna "Soup" é o resultado das capturas de HTMLs pelo requests e o processamento destes pelo BeautifulSoup

A coluna "Tempo" representa o tempo de demora de processamento de cada célula da coluna "Soup", em segundos

In [2]:
#importando a base de ações

AÇ=pd.read_excel("Ações.xlsx")


#AÇ=AÇ.iloc[:5,:] #retirar após testes (termo utilizado para diminuir a quantidade de ações para 5)

#criando coluna do texto da ação no URL
AÇ['N_URL']=[AÇ.iloc[:,1][i]+".SA" if j == "Brasileira" else AÇ.iloc[:,1][i] for i,j in enumerate(AÇ.iloc[:,0])]

#criando coluna do texto da no URL no Yahoo
AÇ['URL']="https://uk.finance.yahoo.com/quote/"+AÇ.N_URL+"/history?period1=1309305600&period2=1685577600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"

#capturando as requests e transformando em soup
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

Soup=[]
Tempo=[]
for url in AÇ.URL:
    Inicio=datetime.now()
    Soup+=[BeautifulSoup(requests.get(url,headers=headers).text, 'html.parser')]
    Fim=datetime.now()
    Tempo+=[(Fim-Inicio).total_seconds()]
AÇ['Soup']=Soup
AÇ['Tempo']=Tempo

## Processamento dos dados



Nesta etapa de processamento dos dados, utilizamos a manipulação de strings para coletar informações sobre ações. 

Percorremos cada elemento da lista de ações, capturando e tratando os dados relevantes. 

Criamos dataframes para armazenar as métricas das ações e os dividendos associados, concatenando-os aos dataframes principais. 

Também identificamos e registramos os elementos atípicos ou com formato de dados inesperado. 

Ao final, temos os dataframes "Métricas", "Dividendos" e "rejeitos". Essa etapa é essencial para extrair informações das páginas web e organizar os dados para análises futuras.

In [3]:
#split duplo
def splitduplo(termo1, termo2, string):
    return string.split(termo1)[1].split(termo2)[0]


#lista de qualquer formato para lista de strings
def listastr(lista):
    return [str(s).replace(",","") for s in list(lista)]

#função principaç
def trazervalores(AÇ):
    Métricas = pd.DataFrame()
    Dividendos = pd.DataFrame()
    rejeitos = pd.DataFrame(columns=['Elemento', 'Ação'])
    
    # Loop para percorrer cada linha do DataFrame AÇ
    for N in range(len(AÇ.loc[:, 'Soup'])):
        # Encontra todos os elementos com a classe "BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)" no objeto Soup
        elements = AÇ.loc[N, 'Soup'].find_all(class_="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)")
        
        # Loop para percorrer cada elemento encontrado
        for i in elements:
            Inicio=datetime.now()
    # Verifica se o elemento possui 7 tags "span", que é o tamanho da tabela no Yahoo finance que tem as métricas das ações
            if len(i.find_all("span")) == 7:
                informações1 = listastr(i)
                
                # Extrai os valores utilizando a função splitduplo
                Data =  datetime.strptime(splitduplo("<span>", "</span>", informações1[0]),"%d %b %Y")
                Open = float(splitduplo("<span>", "</span>", informações1[1]))
                High = float(splitduplo("<span>", "</span>", informações1[2]))
                Low = float(splitduplo("<span>", "</span>", informações1[3]))
                Close = float(splitduplo("<span>", "</span>", informações1[4]))
                AdjClose = float(splitduplo("<span>", "</span>", informações1[5]))
                Volume = float(splitduplo("<span>", "</span>", informações1[6]))
                Fim=datetime.now()
                # Cria um DataFrame temporário com as informações extraídas
                MétricasTemp = pd.DataFrame({
                    'Data': [Data],
                    "Open": [Open],
                    "High": [High],
                    "Low": [Low],
                    "Close": [Close],
                    "AdjClose": [AdjClose],
                    "Volume": [Volume],
                    "Tempo":[(Fim-Inicio).total_seconds()],
                    "Ação": [AÇ.loc[N, 'Ação']]
                })
                
                # Concatena o DataFrame temporário com o DataFrame Métricas
                Métricas = pd.concat([Métricas, MétricasTemp], ignore_index=True)
            
            # Verifica se o elemento possui 2 tags "span", que é o tamanho da linha quando há dividendos no dia
            elif len(i.find_all("span")) == 2:
                try:
                    informações2 = listastr(i)

                    # Extrai os valores utilizando a função splitduplo
                    DataDoDividendo = datetime.strptime(splitduplo("<span>", "</span>", informações1[0]),"%d %b %Y")
                    ValorDoDividendo = splitduplo("<strong>", "</strong>", informações2[1])
                    Classe=splitduplo("</strong>", "</span>", informações2[1]).replace("<span>","")
                    
                    # Cria um DataFrame temporário com as informações extraídas
                    DividendosTemp = pd.DataFrame({
                        'Data': [DataDoDividendo],
                        "Valor do dividendo": [ValorDoDividendo],
                        "Classe":[Classe],
                        "Ação": [AÇ.loc[N, 'Ação']]
                        
                    })

                    # Concatena o DataFrame temporário com o DataFrame Dividendos
                    Dividendos = pd.concat([Dividendos, DividendosTemp], ignore_index=True)
                    
                    
                #célula para identificar erros
                except Exception as e:
                    print(AÇ.loc[N, 'Ação'])
                    print(e)
                    print((AÇ.loc[N, 'URL']))
            # Caso o elemento não se encaixe nos casos anteriores, é considerado um "rejeito" e é adicionado ao DataFrame rejeitos
            else:
                
                #Dataframe com pontos discrepantes
                rejeitos = pd.concat([rejeitos, pd.DataFrame([[i, AÇ.loc[N, 'Ação']]], columns=['Elemento', 'Ação'])])
    return Métricas,Dividendos,rejeitos


Métricas,Dividendos,rejeitos=trazervalores(AÇ)
PF=datetime.now()

In [6]:
contagem_B_ou_E = AÇ['B_ou_E'].value_counts()
contagem_Setor = AÇ['Setor'].value_counts()
estatisticas_numericas = AÇ.describe()

print('Duração total do processo:')
print(PF-IP)


print("Análise Exploratória - DataFrame 1")
print("-----------------------------")
print("Contagem de Valores em 'B_ou_E':")
print(contagem_B_ou_E)
print("\nContagem de Valores em 'Setor':")
print(contagem_Setor)

print("\nMédia do tempo de extração e pré-processamento em segundos:")
print(AÇ['Tempo'].mean())
print("\n")

print("\nEstatísticas Descritivas:")
print(estatisticas_numericas)
print("\n")

# DataFrame 2
duplicados_Ação = Métricas.isnull().count()


print("Análise Exploratória - DataFrame 2")
print("-----------------------------")
print("Número de Valores nulos em 'Ação':", duplicados_Ação)
print("Data do ínicio do período de análise:", Métricas['Data'].min())
print("Data do final do período de análise", Métricas['Data'].max())
print("\n")
print("\nMédia do tempo de extração e pré-processamento em segundos:")
print(Métricas[['Ação','Tempo']].groupby('Ação').sum().mean())
print("\n")

# DataFrame 3
valores_nulos = Dividendos.isnull().count()
#media_dividendos = Dividendos[['Classe']==' Dividend']['Valor do dividendo'].astype(float).mean()
ultima_data = Dividendos['Data'].max()

print("Análise Exploratória - DataFrame 3")
print("-----------------------------")
print("Número de Valores Nulos em 'Valor do dividendo':", valores_nulos)
#print("Valor Médio de Dividendos:", media_dividendos)
print("Última Data Registrada:", ultima_data)
print("\n")

# DataFrame 4
num_elementos = len(rejeitos)
num_acoes_exclusivas = rejeitos['Ação'].nunique()

print("Análise Exploratória - DataFrame 4")
print("-----------------------------")
print("Número de Elementos:", num_elementos)
print("Número de Ações Exclusivas:", num_acoes_exclusivas)
print("\n")




Duração total do processo:
0:07:54.781528
Análise Exploratória - DataFrame 1
-----------------------------
Contagem de Valores em 'B_ou_E':
Estrangeira    135
Brasileira      65
Name: B_ou_E, dtype: int64

Contagem de Valores em 'Setor':
Tecnologia             26
Energia                25
Financeiro             22
Saúde                  13
Varejo                 13
Farmacêutico           11
Telecomunicações        8
Indústria               8
Seguros                 7
Imobiliário             7
Entretenimento          5
Aeroespacial            4
Alimentação             4
Bebidas                 4
Bens de Consumo         3
Transporte              3
Papel e Celulose        3
Automotivo              3
Turismo                 3
Químicos                3
Vestuário               3
Locação de Veículos     2
Infraestrutura          2
Siderurgia              2
Alimentos               2
Mineração               2
Tabaco                  2
Serviços de TI          1
Construção Civil        1
Petroquí

## Conclusão

O projeto foi concluído com sucesso, resultando em quatro dataframes. O tempo de execução do processo inteiro foi de 07 minutos e 54 segundos, e, para melhorar o tempo, será necessário modificações no código. A utilização de listas talvez prejudique a performance, assim como a quantidade de loops.

A partir do funcionamento do código, foi possível coletar apenas os dados entre 01/05/2023 e 02/06/2023. Para superar esta barreira e adquirir todos os dados possíveis, será necessário o conhecimento da formação do URL, que estipula o período, e o uso de alguma outra biblioteca, como o Selenium, para alançar os dados que são enviados ao usuário após rolar a página até o fim.

Apesar dos pontos de potenciais melhorias que serão descritos abaixo, o código funciona efetivamente. A análise pode ser mais robusta pois a atual não entrega uma completude no entendimento dos dados.


### Pontos de melhoria

- Otimização do tempo de execução: 

É necessário rever o código e identificar possíveis áreas de melhoria para reduzir o tempo total de execução. A utilização de listas e loops pode afetar a performance, portanto, considerar abordagens mais eficientes pode ajudar a melhorar o tempo de processamento.

- Aquisição de dados completos: 

Identificou-se uma limitação na coleta de dados, onde apenas os registros entre 01/05/2023 e 02/06/2023 foram obtidos. Para superar essa limitação, é recomendado explorar a estrutura do URL utilizado para a coleta de dados e investigar se é possível configurar um período mais abrangente. Além disso, o uso de bibliotecas como o Selenium pode ser necessário para automatizar a rolagem da página e obter todos os dados disponíveis.

- Organização do código: 

Refatorar o código para torná-lo mais modular e legível. Isso inclui separar as funcionalidades em funções reutilizáveis, adicionar comentários explicativos e seguir as boas práticas de codificação. Uma estrutura de código bem organizada facilita a manutenção, depuração e colaboração com outros desenvolvedores.

- Visualização de dados: 

Aprimorar a análise exploratória incorporando visualizações de dados por meio de gráficos. Utilize bibliotecas como Matplotlib, Seaborn ou Plotly para criar gráficos informativos, como gráficos de barras, gráficos de dispersão, histogramas, entre outros. Essas visualizações permitem identificar padrões, tendências e relações nos dados, facilitando a compreensão e comunicação dos resultados. Além disso, considere a criação de gráficos interativos que permitam a exploração dos dados de forma dinâmica. A inclusão desses gráficos irá enriquecer a análise exploratória, tornando-a mais impactante e compreensível para os usuários.